In [2]:
from pydrake.all import *
import sympy as sp

In [3]:
x, y = sp.symbols('x y')

In [4]:
F = 10 - x**2 - y

In [5]:
G = x**2 + y**2 - 1

In [8]:
res = sp.groebner([G], x, y, order='grlex')

In [9]:
res

GroebnerBasis([x**2 + y**2 - 1], x, y, domain='ZZ', order='grlex')

In [29]:
Q = sp.MatrixSymbol('Q', 3, 3)
Q = sp.Matrix(Q)

In [30]:
mx = sp.Matrix([1, x, y])

In [31]:
temp = mx.T@Q@mx

In [32]:
sp.simplify(temp)

Matrix([[x*(x*Q[1, 1] + y*Q[2, 1] + Q[0, 1]) + x*Q[1, 0] + y*(x*Q[1, 2] + y*Q[2, 2] + Q[0, 2]) + y*Q[2, 0] + Q[0, 0]]])

In [79]:
prog = MathematicalProgram()

x = prog.NewIndeterminates(1, 'x')[0]
y = prog.NewIndeterminates(1, 'y')[0]
Q = prog.NewSymmetricContinuousVariables(6, 'Q')

mx = np.hstack([x, y, x**2, x*y, y**2, y*x**2])

expression = Polynomial(mx.T@Q@mx)

original_expression = Polynomial(x**2 + y**2)

prog.AddEqualityConstraintBetweenPolynomials(expression, original_expression)
result = solver.Solve(prog)

In [80]:
result.GetSolution(Q)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [73]:
expression = Polynomial(mx.T@Q@mx)

original_expression = Polynomial(10 - x[0]**2 - y[0])

prog.AddEqualityConstraintBetweenPolynomials(expression, original_expression)
#prog.AddPositiveSemidefiniteConstraint(Q)
solver = MosekSolver()

result = solver.Solve(prog)

TypeError: 'pydrake.symbolic.Variable' object is not subscriptable

In [63]:
result.GetSolution(Q)

array([[10. ,  0. , -0.5],
       [ 0. , -1. ,  0. ],
       [-0.5,  0. ,  0. ]])

In [64]:
mx.T@result.GetSolution(Q)@mx

<Expression "(10 - y(0) - pow(x(0), 2))">

In [65]:
np.linalg.eigvals(result.GetSolution(Q))

array([10.02493781, -0.02493781, -1.        ])

In [68]:
Q_test = np.array([[9, 0, -1/2], [0, 0, 0], [-1/2, 0, 1]])

In [69]:
Polynomial(mx.T@Q_test@mx)

<Polynomial "9*1 + -1*y(0) + 1*y(0)^2">

In [88]:
prog = MathematicalProgram()
S = prog.NewSymmetricContinuousVariables(2, "S")
psd_constraint = prog.AddPositiveSemidefiniteConstraint(S)

#prog.AddBoundingBoxConstraint(1, 1, S[1, 0])
#prog.AddLinearCost(np.trace(S))
result = Solve(prog)

S_value = result.GetSolution(S)

In [89]:
np.linalg.eigvalsh(S_value)

array([0.70710678, 0.70710678])

In [90]:
S_value

array([[0.70710678, 0.        ],
       [0.        , 0.70710678]])